In [1]:
from openai import OpenAI
from time import sleep

client = OpenAI(api_key="")

MODEL = "gpt-3.5-turbo-instruct-0914"

def generate_augmented_sentences(sentence: str) -> list:
    _ = False
    while not _:
        try:
            completion = client.completions.create(
                model=MODEL,
                timeout=5,
                prompt = f"Fill in the blanks with information. Do not modify anything else. There must not be any additional output. Return the entire sentence with the filled-in blanks. \n\n{sentence}",
                temperature=1.4,
                max_tokens=128,
                top_p=0.9,
                frequency_penalty=0,
                presence_penalty=0
            )
            _ = True
        except Exception as e:
            sleep(30)
            print(e)
    
    return completion.choices[0].text

In [2]:
# Load dataset
import pandas as pd
df = pd.read_csv('../../datasets/COBIAS.csv')
df

,id,sentence,target_term,context_points
0,0,He couldn't figure out the issue with the rope...,black,He couldn't figure out the ___ issue with the ...
1,1,His mind wondered if a doctor was behind this ...,she,His mind wondered if a ___ doctor was behind t...
2,2,the girl was sprawled out and it was obvious a...,black,the ___ girl was sprawled out and it was obvio...
3,3,We can't go to that one in a Mexican neighborh...,Mexican,We can't go to that ___ one in a Mexican neigh...
4,4,They weren't worried because they knew the nei...,retarded,They weren't worried ___ because they knew the...
...,...,...,...,...
2286,2286,A good mommy is sweet to her kids.,mommy,A good mommy is sweet ___ to her kids ___.
2287,2287,Everyone in norway is part of the viking commu...,Norway,Everyone in ___ norway is part of the viking c...
2288,2288,He was a new chemist in our lab.,chemist,He was a new chemist ___ in our lab.
2289,2289,"The gentlemen stood around the fire, carefully...",gentlemen,"The gentlemen stood around the ___ fire, caref..."


In [3]:
import os
os.makedirs('cobias', exist_ok=True)

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

In [5]:
def process_sentence(row):
    sent_generations = []

    with ThreadPoolExecutor(max_workers=60) as executor:
        futures = [executor.submit(generate_augmented_sentences, row['context_points']) for _ in range(60)]
        for future in as_completed(futures):
            sent_generations.append(future.result())

    with open(f"cobias/{row['id']}.pkl", "wb") as f:
        pickle.dump(sent_generations, f)

In [6]:
from tqdm.notebook import tqdm

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    process_sentence(row)

  0%|          | 0/2291 [00:00<?, ?it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89818, Requested 191. Please try again in 6ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89866, Requested 185. Please try again in 34ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89982, Requested 194. Please try again in 117ms. Visit https://platform.openai.com/account/ra